In [9]:
# Import required Libraries
import pandas as pd
import os
from pandas import json_normalize
import json
import requests
from IPython.display import JSON

### Import Test Sample

In [10]:
flight_sample = pd.read_csv('./Data/Flight_test_sample.csv')

### Weather Functions

In [11]:
# Weather API Fetch Function
def world_weather_fetch(query):
    
        # Set Key
        weather_key = os.environ['weather_api_key']
        
        # Set Parameters for API
        weather_url = "http://api.worldweatheronline.com/premium/v1/weather.ashx?"
        location = query
        weather_response = requests.get(weather_url,params={
                                                  'q':location,
                                                  'date':'2020-01-01',
                                                  'fx':'no',
                                                  'cc':'no',
                                                  'mca':'yes',
                                                  'format':'json',
                                                  'key':weather_key})
        
        # Call API and direct returned information
        weather_result = weather_response.json()
        df_weather = json_normalize(weather_result)
        return weather_result, df_weather

In [12]:
flight_sample = flight_sample.drop(columns=['avgThunderDays','avgRainDays','avgDryDays','avgCloud','avgSnowDays'])

In [5]:
flight_sample.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
       'flight_speed', 'flight_haul_type', 'flight_month',
       'arrival_hour_of_day', 'departure_hour_of_day', 'percent_delay',
       'percent_flights', 'avg_flights', 'avg_passengers', 'total_distance',
       'total_passengers', 'avg_fuel', 'state'],
      dtype='object')

In [13]:
locations = list(flight_sample.dest_city_name.unique())

In [20]:
locations

['San Francisco, CA',
 'Sacramento, CA',
 'Phoenix, AZ',
 'Denver, CO',
 'San Jose, CA',
 'Las Vegas, NV',
 'Dallas, TX',
 'Portland, OR',
 'Chicago, IL',
 'Oakland, CA',
 'Dallas/Fort Worth, TX',
 'Salt Lake City, UT',
 'Atlanta, GA',
 'Austin, TX',
 'Orlando, FL',
 'Seattle, WA',
 'New York, NY',
 'Houston, TX',
 'Albuquerque, NM',
 'Tucson, AZ',
 'Boise, ID',
 'Baltimore, MD',
 'Omaha, NE',
 'St. Louis, MO',
 'Nashville, TN',
 'Reno, NV',
 'San Antonio, TX',
 'Spokane, WA',
 'Tampa, FL',
 'Kansas City, MO',
 'El Paso, TX',
 'New Orleans, LA',
 'Charlotte, NC',
 'Philadelphia, PA',
 'Miami, FL',
 'Los Angeles, CA',
 'Detroit, MI',
 'Minneapolis, MN',
 'Fresno, CA',
 'Monterey, CA',
 'Santa Rosa, CA',
 'San Luis Obispo, CA',
 'Hayden, CO',
 'Newark, NJ',
 'Washington, DC',
 'Kahului, HI',
 'Kona, HI',
 'Honolulu, HI',
 'Boston, MA',
 'Lihue, HI',
 'Fort Lauderdale, FL',
 'Everett, WA',
 'Bend/Redmond, OR',
 'Burbank, CA',
 'San Diego, CA',
 'Ontario, CA',
 'Cincinnati, OH',
 'Oklahoma

In [25]:
def compile_weather_data(string_list):
    
    columns=['avgRainDays','avgDryDays','avgCloud','avgSnowDays']
    
    # Call Api for 1st entry in list and establish dataframe to build upon
    init_result,init_data = world_weather_fetch(string_list[0])
    init_data = json_normalize(data=init_data['data.ClimateAverages'][0][0]['month'][0])
    # Modify dataframe types for specified columns
    init_data = init_data[columns].apply(pd.to_numeric)
    # Add location data to initial dataframe 
    init_data['location'] = string_list[0]
    
    for city in string_list[1:]:
        #Call weather API per location and get dataframe values
        result,data = world_weather_fetch(city)
        data = json_normalize(data=data['data.ClimateAverages'][0][0]['month'][0])
        # Modify dataframe types for specified columns
        data = data[columns].apply(pd.to_numeric)
        # Add location data to dataframe 
        data['location'] = city
        # Append Dataframe to initial dataframe
        init_data = init_data.append(data)
  
    return init_data

### Get Weather Data for Each Destination Airport

In [26]:
weather_data = compile_weather_data(locations)

KeyError: "None of [Index(['avgRainDays', 'avgDryDays', 'avgCloud', 'avgSnowDays'], dtype='object')] are in the [columns]"

In [87]:
weather_data = weather_data[['location','avgDryDays','avgRainDays','avgCloud','avgSnowDays']]
weather_data = weather_data.rename(columns={"location": "dest_city_name"})

In [110]:
flight_sample = flight_sample.drop(columns=['dest_city_name'])
flight_sample = pd.merge(flight_sample,weather_data,how='outer',left_index=True,right_index=True)

In [115]:

import pickle
outfile = './Data/Flight_2020_Test_sample'
weather_2020_test = open(outfile,'wb')
pickle.dump(flight_sample,weather_2020_test)
weather_2020_test.close()